# Path

In [1]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Import

In [2]:
from helpers import column_formatter, utf8_convert
from plotly.subplots import make_subplots
from charset_normalizer import detect
from config import data_path as path
from itertools import combinations 
import plotly.graph_objects as go
from datetime import datetime
from json import dump, load
import plotly.express as px
import pandas as pd
import warnings
import openpyxl

# Read

In [3]:
information = pd.read_excel(
    path+'/3-result/information.xlsx', index_col='Unnamed: 0')
events_per_person = pd.read_excel(
    path+'/3-result/events_per_person.xlsx', index_col='Unnamed: 0')
events = pd.read_excel(
    path+'/3-result/events.xlsx', index_col='Unnamed: 0')
ohe = pd.read_excel(
    path+'/3-result/event_per_person_ohe.xlsx', index_col='Unnamed: 0')


with pd.ExcelFile(path + '/2-cleaned/basvuru.xlsx') as appeals_xlsx:
    appeals = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Başvurular')
    lessons_all = pd.read_excel(
        appeals_xlsx, index_col='Unnamed: 0', sheet_name='Dersler')


with pd.ExcelFile(path + '/2-cleaned/yoklama.xlsx') as rolls_xlsx:
    rolls_info = pd.read_excel(
        rolls_xlsx, 'General Info', index_col='Unnamed: 0')

    rolls = dict()
    for sheet in rolls_info.index:
        rolls[sheet] = pd.read_excel(rolls_xlsx, sheet)

information['id'] = information.index

# Range

In [4]:
ohe_appeals = ohe[ohe.columns[::2]].copy()
ohe_rolls = ohe[ohe.columns[1::2]].copy()
ohe_appeals.loc[:, 'sum'] = ohe_appeals.T.sum()
ohe_rolls.loc[:, 'sum'] = ohe_rolls.T.sum()

## Range Mean

In [5]:
for i in ((0,5), (5,10), (10, float('inf'))):
    print(f"[{i[0]}, {i[1]}) arasında başvuru yapanların ortalamaları.",
          f"rolls mean : {ohe_rolls[(ohe_appeals['sum'] >= i[0]) & (ohe_appeals['sum'] < i[1])]['sum'].mean():.2f}",
          f"appeals mean : {ohe_appeals[(ohe_appeals['sum'] >= i[0]) & (ohe_appeals['sum'] < i[1])]['sum'].mean():.2f}",
          sep='\n', end='\n\n')

[0, 5) arasında başvuru yapanların ortalamaları.
rolls mean : 0.59
appeals mean : 1.99

[5, 10) arasında başvuru yapanların ortalamaları.
rolls mean : 0.93
appeals mean : 6.70

[10, inf) arasında başvuru yapanların ortalamaları.
rolls mean : 0.99
appeals mean : 12.49



# Zero Appeals

In [6]:
zero_appeals_events = ohe_rolls[(ohe_appeals['sum'] == 0) & (ohe_rolls['sum'] != 0)].sum()
pd.DataFrame(zero_appeals_events[zero_appeals_events != 0].sort_values(ascending=False))

# k13 toplam katılım :25, k15 toplam katılım 26

,0
sum,62
k13,15
k15,13
k11,7
k5,6
k8,5
k12,4
k20,4
k16,3
k9,2


In [7]:
list(enumerate(events['eğitim'], start=1))

[(1, 'Hikaye Anlatıcılığı 101'),
 (2, 'Finans 101'),
 (3, 'Büyük Veriye Giriş 101'),
 (4, 'Proje Döngüsü Yönetimi 101'),
 (5, 'Girişimcilik 101'),
 (6, 'Arduino 101'),
 (7, 'İşletim Sistemleri 101'),
 (8, 'IOT 101'),
 (9, 'Zihin Haritalama 101'),
 (10, 'Blockchain 101'),
 (11, 'Siber Güvenlik 101'),
 (12, 'Eleştirel Düşünce 101'),
 (13, 'Yapay Zeka 101'),
 (14, 'Python 101'),
 (15, 'İnovasyon 101'),
 (16, 'Machine Learning 101'),
 (17, 'Dijital Şehir 101'),
 (18, 'Dijital Dünyada Bilinç 101'),
 (19, 'Tasarım Odaklı Düşünce 101'),
 (20, 'Araştırma Yöntemleri 101'),
 (21, 'Tasarım İlkeleri 101'),
 (22, 'Tasarım 101'),
 (23, 'Metaverse 101'),
 (24, 'Programlama ve Algoritma 101')]

# Max Ratio

In [8]:
ratio = pd.DataFrame(ohe_rolls['sum'] / ohe_appeals['sum'])
ratio = ratio.sort_values(by='sum', ascending=False)
ratio = ratio[ratio['sum'] != float('inf')].dropna()
ratio.dropna(inplace=True)

In [9]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Katılım Box Plot', 'Katılım Dağılımı'))
fig.add_trace(go.Box(x=ratio['sum']), row=1, col=1)
fig.add_trace(go.Histogram(x=ratio['sum']), row=1, col=2)
fig.update_layout(showlegend=False, height=300, width=1200, title_text="Katılım Oranı")

In [10]:
print(ratio[ratio['sum'] >= 0.10].shape)
print(ratio[ratio['sum'] >= 0.20].shape)
print(ratio[ratio['sum'] >= 0.50].shape)

(160, 1)
(118, 1)
(49, 1)


In [11]:
big50 = ohe.T[ratio[ratio['sum'] >= 0.50].index].T.copy()
big50['sum_b'] = big50[big50.columns[::2]].T.sum()
big50['sum_k'] = big50[big50.columns[1::2]].T.sum()
print(big50['sum_b'].mean(), big50['sum_k'].mean())

3.8979591836734695 2.5510204081632653


In [12]:
high_ratio_rolls = ratio[ratio['sum'] >= 0.5].index
high_ratio_rolls = ohe_rolls.T[high_ratio_rolls].T.sum()
pd.DataFrame(high_ratio_rolls[high_ratio_rolls != 0].sort_values(ascending=False))

,0
sum,125
k11,15
k5,9
k19,9
k15,9
k14,8
k4,8
k9,8
k21,6
k18,6
